### Spark_The_Definitive_Guide chapter02.overview spark
> 이해 부족 추가공부 필요 <br>
> - 2.7 트랜스포메이션, 셔플, 파이프라이닝 <br>
> - 2.10 실행계획 파트 이해 부족 



In [46]:
#import os
#os.environ['SPARK_HOME'] = '/Users/jeongmo/.sdkman/candidates/spark/current/'
#os.environ['SPARK_LOCAL_IP'] = '127.0.0.1'
#os.system("pyspark -c spark.driver.bindAddress=127.0.0.1")
#from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
spark = (
    SparkSession
    .builder
    .appName("sparkJM")
    .master("local[1]")
    .config("spark.driver.host","127.0.0.1") #spark.driver.bindAdress Error solve
    .config("spark.driver.bindAddress","127.0.0.1")  #spark.driver.bindAdress Error solve
    .getOrCreate()
)


In [3]:
# spark 는 데이터를 읽어 로컬의 리스트/배열 형태로 변환함
sdf = spark\
    .read\
    .option("inferSchema","True")\
    .option("header","True")\
    .csv("/Users/jeongmo/VSCodeLib/Study/data_from_git/Spark-The-Definitive-Guide/data/flight-data/csv/")

In [4]:
# take 메서드를 쓰면 해당 내용을 볼 수 있음
# explain 메서드 쓰면 spark 의 실행계획 볼 수 있음
print(sdf.take(3))
sdf.sort("count").explain()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1), Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=264), Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=69)]
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#19 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#19 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=33]
      +- FileScan csv [DEST_COUNTRY_NAME#17,ORIGIN_COUNTRY_NAME#18,count#19] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/jeongmo/VSCodeLib/Study/data_from_git/Spark-The-Definitive..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [5]:
# spark 는 셔플 수행시, 기본적으로 200개의 셔플 파티션을 생성함. 셔플파티션 수 지정 옵션
spark.conf.set("spark.sql.shuffle.partitions", "5")  
sdf.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#19 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#19 ASC NULLS FIRST, 5), ENSURE_REQUIREMENTS, [plan_id=43]
      +- FileScan csv [DEST_COUNTRY_NAME#17,ORIGIN_COUNTRY_NAME#18,count#19] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/jeongmo/VSCodeLib/Study/data_from_git/Spark-The-Definitive..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [6]:
print(sdf.printSchema())
print(sdf.count())
sdf.show()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)

None
1502
+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            

In [36]:
# sparkDataFrame 을 임시테이블(view)로 만들고, SQL 사용 가능!
# spatkSQL 은 sparkDataFrame 코드와 같은 실행계획으로 컴파일 함 -> 둘 사이의 성능차이 없음
sdf.createOrReplaceTempView("sdf_ex")
sdf_by_sql = spark.sql(
    """
    SELECT DEST_COUNTRY_NAME, COUNT(1) as CNT
    FROM sdf_ex
    WHERE 1=1
    GROUP BY DEST_COUNTRY_NAME
    ORDER BY DEST_COUNTRY_NAME
    """
    )

sdf_1 = sdf.groupby("DEST_COUNTRY_NAME")\
    .count()\
    .withColumnRenamed("count","CNT")\
    .orderBy("DEST_COUNTRY_NAME")

In [37]:
sdf_1.show()

+--------------------+---+
|   DEST_COUNTRY_NAME|CNT|
+--------------------+---+
|         Afghanistan|  3|
|             Algeria|  3|
|              Angola|  6|
|            Anguilla|  6|
| Antigua and Barbuda|  6|
|           Argentina|  6|
|               Aruba|  6|
|           Australia|  6|
|             Austria|  6|
|          Azerbaijan|  4|
|             Bahrain|  6|
|            Barbados|  6|
|             Belarus|  2|
|             Belgium|  6|
|              Belize|  6|
|             Bermuda|  6|
|             Bolivia|  6|
|Bonaire, Sint Eus...|  6|
|              Brazil|  6|
|British Virgin Is...|  6|
+--------------------+---+
only showing top 20 rows



In [38]:
print(sdf_by_sql.explain())
print(sdf_1.explain())

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [DEST_COUNTRY_NAME#17 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(DEST_COUNTRY_NAME#17 ASC NULLS FIRST, 5), ENSURE_REQUIREMENTS, [plan_id=559]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[count(1)])
         +- Exchange hashpartitioning(DEST_COUNTRY_NAME#17, 5), ENSURE_REQUIREMENTS, [plan_id=556]
            +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[partial_count(1)])
               +- FileScan csv [DEST_COUNTRY_NAME#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/jeongmo/VSCodeLib/Study/data_from_git/Spark-The-Definitive..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


None
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [DEST_COUNTRY_NAME#17 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(DEST_COUNTRY_NAME#17 ASC NULLS FIRST, 5), ENSURE_REQUIREMENTS, [pla

In [45]:
sdf\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|          2348280|
|           Canada|            49052|
|           Mexico|            38075|
|   United Kingdom|            10946|
|            Japan|             9205|
+-----------------+-----------------+



In [41]:
sdf\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#371L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#17,destination_total#371L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[sum(count#19)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#17, 5), ENSURE_REQUIREMENTS, [plan_id=640]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[partial_sum(count#19)])
            +- FileScan csv [DEST_COUNTRY_NAME#17,count#19] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/jeongmo/VSCodeLib/Study/data_from_git/Spark-The-Definitive..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


